### 프로젝트 3

[주제] 
일정 범위 내에 존재하는 객체 탐지하기

[구현 기능]
1. 웹 캠 열기:
2. 화면 내 원하는 영역을 도형으로 표시
3. 도현 안에서 탐지되는 객체들을 우측 리스트에 표시

[추가 기능]
1. 도형 밖으로 객체가 나갈 경우 리스트에서 제외

In [1]:
# 라이브러리 불러오기
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [2]:
# YOLO 모델 불러오기
model = YOLO('yolov8n.pt')
model.info()

YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(129, 3157200, 0, 8.8575488)

In [3]:
# 객체의 중심이 원 안에 있는지 확인하는 함수
def is_center_in_circle(box, circle_center, circle_radius):
    """객체 바운딩 박스의 중심이 원 내부에 있는지 확인합니다."""
    x1, y1, x2, y2 = map(int, box)
    box_center_x = (x1 + x2) / 2
    box_center_y = (y1 + y2) / 2
    
    # 원의 중심과 객체 중심 사이의 거리 계산
    distance = math.sqrt((box_center_x - circle_center[0])**2 + (box_center_y - circle_center[1])**2)
    
    return distance <= circle_radius

def main():
    # YOLOv8 모델 로드
    model = YOLO('yolov8n.pt')

    # 웹캠 열기
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("웹캠을 열 수 없습니다.")
        return

    # 웹캠 프레임 크기 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 감지 영역으로 사용할 원 정의 (프레임 중앙)
    circle_center = (width // 2, height // 2)
    circle_radius = 150

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 매 프레임마다 원 안의 객체 목록을 초기화
        objects_in_circle = []

        # YOLO로 객체 감지
        results = model(frame)
        
        # 기준이 되는 원 그리기
        cv2.circle(frame, circle_center, circle_radius, (255, 255, 0), 2)

        # 감지된 객체 처리 (Ultralytics v8 결과 형식에 맞게 수정)
        for box in results[0].boxes:
            # 바운딩 박스 좌표
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # 객체 클래스 및 신뢰도
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            class_name = model.names[cls]

            # 객체가 원 안에 있는지 확인
            is_inside = is_center_in_circle(box.xyxy[0], circle_center, circle_radius)
            
            # 박스 색상 설정 (원 안에 있으면 빨간색)
            color = (0, 0, 255) if is_inside else (0, 255, 0)
            
            # 바운딩 박스 그리기
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # 객체 레이블 표시
            label = f"{class_name} {conf:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            # 원 안에 있는 객체 처리
            if is_inside:
                # 클래스 이름을 목록에 추가
                objects_in_circle.append(class_name)
                
                # 경고 메시지 출력
                cv2.putText(frame, "Object in Zone!", 
                          (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                          1, (0, 0, 255), 2)

        # 원 안에 있는 객체들의 클래스 이름을 우측 상단에 표시
        y_offset = 50
        # 중복을 제거하고 정렬하여 표시
        for name in sorted(list(set(objects_in_circle))):
            text = f"In Zone: {name}"
            cv2.putText(frame, text, (width - 200, y_offset), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            y_offset += 30

        # 결과 화면 표시
        cv2.imshow('Object Detection', frame)

        # 'ESC' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # 자원 해제
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()



0: 480x640 1 person, 76.6ms
Speed: 1.3ms preprocess, 76.6ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 67.6ms
Speed: 1.7ms preprocess, 67.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 52.0ms
Speed: 1.4ms preprocess, 52.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 55.0ms
Speed: 1.3ms preprocess, 55.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 53.5ms
Speed: 1.3ms preprocess, 53.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 54.5ms
Speed: 3.2ms preprocess, 54.5ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 55.6ms
Speed: 1.3ms preprocess, 55.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 51.7ms
Speed: 1.1ms preprocess, 51.7ms inference, 0.9ms postprocess per image at shape (1, 3, 48